# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [26]:
!pip3 install nltk

In [27]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from newsapi import NewsApiClient
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\rlkee\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [28]:
# Read your api key environment variable
# Load environment variables 
load_dotenv()


True

In [29]:
api_key = os.getenv("NEWS_API")
newsapi = NewsApiClient(api_key="70f4b1a8cfea46418e734d354b103dbe")

In [30]:
# Fetch the Bitcoin news articles
btc_headlines = newsapi.get_everything(
    q="bitcoin",
    language="en",
    sort_by="relevancy"
)

print(f"Total articles about btc: {btc_headlines['totalResults']}")

# Show sample article
btc_headlines["articles"][0]


Total articles about btc: 7788


{'source': {'id': None, 'name': 'Blogspot.com'},
 'author': 'noreply@blogger.com (Unknown)',
 'title': "Elon Musk reveals who bitcoin's creator Satoshi Nakamoto might be",
 'description': 'Musk.MARK RALSTON/AFP via Getty Images\r\nElon Musk seems to agree with many that hyper-secret cryptocurrency expert Nick Szabo could be Satoshi Nakamoto, the mysterious creator of the digital currency Bitcoin.\xa0"You can watch ideas evolve before Bitcoin was lau…',
 'url': 'https://techncruncher.blogspot.com/2021/12/elon-musk-reveals-who-bitcoins-creator.html',
 'urlToImage': 'https://blogger.googleusercontent.com/img/a/AVvXsEik_48hPzMzsDzwfdUeHj4jNGqYGevEuVjTTPkAGTu9bRN3oePxV6bxF897GK8Az3AaSqUOalYXNG4HSCy0fW5KUHuruCWP8hAfZxgrgbzh-dsbLM9jxyFGCthOZdBCa1dNkqk6mrVl0VtflsV2VvKXfGnwL6-68m-mxp7qHJuLlvqGIahZ9YDe5mt97w=w1200-h630-p-k-no-nu',
 'publishedAt': '2021-12-29T20:41:00Z',
 'content': 'Musk.MARK RALSTON/AFP via Getty Images\r\nElon Musk seems to agree with many that hyper-secret cryptocurrency expe

In [31]:
# Fetch the Ethereum news articles
eth_headlines = newsapi.get_everything(
    q="Ethereum",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about eth: {eth_headlines['totalResults']}")

# Show sample article
eth_headlines["articles"][0]


Total articles about eth: 3547


{'source': {'id': 'the-verge', 'name': 'The Verge'},
 'author': 'Corin Faife',
 'title': 'Crypto.com admits over $30 million stolen by hackers',
 'description': 'Cryptocurrency exchange Crypto.com has said that $15 million in ethereum and $18 million in bitcoin were stolen by hackers in a security breach',
 'url': 'https://www.theverge.com/2022/1/20/22892958/crypto-com-exchange-hack-bitcoin-ethereum-security',
 'urlToImage': 'https://cdn.vox-cdn.com/thumbor/mde_l3lUC4muDPEFG7LYrUz0O3g=/0x146:2040x1214/fit-in/1200x630/cdn.vox-cdn.com/uploads/chorus_asset/file/8921023/acastro_bitcoin_2.jpg',
 'publishedAt': '2022-01-20T13:23:31Z',
 'content': 'In a new blog post the company said that 4,836 ETH and 443 bitcoin were taken\r\nIllustration by Alex Castro / The Verge\r\nIn a blog post published in the early hours of Thursday morning, cryptocurrency… [+2004 chars]'}

In [32]:
# Create the Bitcoin sentiment scores DataFrame
btc_sentiments = []

for article in btc_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        btc_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
btc_df = pd.DataFrame(btc_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
btc_df = btc_df[cols]

btc_df.head()



,date,text,compound,positive,negative,neutral
0,2021-12-29,Musk.MARK RALSTON/AFP via Getty Images\r\nElon...,0.3612,0.077,0.000,0.923
1,2022-01-12,When Denis Rusinovich set up cryptocurrency mi...,0.0000,0.000,0.000,1.000
2,2022-01-25,El Salvador introduced Bitcoin as a legal tend...,0.3182,0.105,0.000,0.895
3,2022-01-14,Were officially building an open Bitcoin minin...,-0.4404,0.000,0.083,0.917
4,2022-01-27,Israeli national Tal Prihar pled guilty to rou...,-0.3182,0.045,0.084,0.871


In [33]:
# Create the Bitcoin sentiment scores DataFrame
btc_df = pd.DataFrame.from_dict(btc_headlines["articles"])

btc_df.head()

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'Blogspot.com'}",noreply@blogger.com (Unknown),Elon Musk reveals who bitcoin's creator Satosh...,Musk.MARK RALSTON/AFP via Getty Images\r\nElon...,https://techncruncher.blogspot.com/2021/12/elo...,https://blogger.googleusercontent.com/img/a/AV...,2021-12-29T20:41:00Z,Musk.MARK RALSTON/AFP via Getty Images\r\nElon...
1,"{'id': 'wired', 'name': 'Wired'}",Gian M. Volpicelli,"As Kazakhstan Descends into Chaos, Crypto Mine...",The central Asian country became No. 2 in the ...,https://www.wired.com/story/kazakhstan-cryptoc...,https://media.wired.com/photos/61de2d453e654a1...,2022-01-12T12:00:00Z,When Denis Rusinovich set up cryptocurrency mi...
2,"{'id': 'the-verge', 'name': 'The Verge'}",Mitchell Clark,The International Monetary Fund tells El Salva...,The International Monetary Fund’s executive di...,https://www.theverge.com/2022/1/25/22901374/el...,https://cdn.vox-cdn.com/thumbor/altkKN7BnaLUpb...,2022-01-25T22:11:14Z,El Salvador introduced Bitcoin as a legal tend...
3,"{'id': 'the-verge', 'name': 'The Verge'}",Jon Porter,Jack Dorsey’s Block is working to make Bitcoin...,Block is working on building an “open Bitcoin ...,https://www.theverge.com/2022/1/14/22883500/ja...,https://cdn.vox-cdn.com/thumbor/NE548fVufAlHoB...,2022-01-14T13:46:28Z,Were officially building an open Bitcoin minin...
4,"{'id': 'the-verge', 'name': 'The Verge'}",Corin Faife,DeepDotWeb operator sentenced to eight years f...,"The operator of DeepDotWeb, a site that indexe...",https://www.theverge.com/2022/1/27/22904803/de...,https://cdn.vox-cdn.com/thumbor/mde_l3lUC4muDP...,2022-01-27T18:16:57Z,Israeli national Tal Prihar pled guilty to rou...


In [34]:
# Create the Ethereum sentiment scores DataFrame
eth_sentiments = []

for article in eth_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        eth_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
eth_df = pd.DataFrame(eth_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
eth_df = eth_df[cols]

eth_df.head()


,date,text,compound,positive,negative,neutral
0,2022-01-20,"In a new blog post the company said that 4,836...",0.0000,0.000,0.000,1.000
1,2022-01-19,Hackers who made off with roughly $15 million ...,0.0000,0.000,0.000,1.000
2,2022-01-20,"On some level, the new mayor is simply employi...",0.1779,0.052,0.000,0.948
3,2022-01-21,"Back in September\r\n, Robinhood announced pla...",0.0772,0.038,0.000,0.962
4,2022-01-20,Trading platform Crypto.com lost about $34 mil...,-0.1027,0.056,0.067,0.877


In [45]:
# Create the Ethereum sentiment scores DataFrame
eth_df = pd.DataFrame.from_dict(eth_headlines["articles"])

eth_df.head()

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'the-verge', 'name': 'The Verge'}",Corin Faife,Crypto.com admits over $30 million stolen by h...,Cryptocurrency exchange Crypto.com has said th...,https://www.theverge.com/2022/1/20/22892958/cr...,https://cdn.vox-cdn.com/thumbor/mde_l3lUC4muDP...,2022-01-20T13:23:31Z,"In a new blog post the company said that 4,836..."
1,"{'id': None, 'name': 'Gizmodo.com'}",Matt Novak,Hackers Launder $15 Million Stolen From Crypto...,Hackers who made off with roughly $15 million ...,https://gizmodo.com/hackers-launder-15-million...,https://i.kinja-img.com/gawker-media/image/upl...,2022-01-19T12:00:00Z,Hackers who made off with roughly $15 million ...
2,"{'id': None, 'name': 'New York Times'}","Dana Rubinstein, Jonah E. Bromwich and Jeffery...",Eric Adams Is Taking His First Paycheck in Crypto,"Mr. Adams, who wants New York City to become t...",https://www.nytimes.com/2022/01/20/nyregion/er...,https://static01.nyt.com/images/2022/01/20/nyr...,2022-01-20T19:54:48Z,"On some level, the new mayor is simply employi..."
3,"{'id': 'engadget', 'name': 'Engadget'}",Igor Bonifacic,Robinhood opens cryptocurrency wallet to beta ...,"Back in September\r\n, Robinhood announced pla...",https://www.engadget.com/robinhood-cryptocurre...,https://s.yimg.com/os/creatr-uploaded-images/2...,2022-01-21T22:57:21Z,"Back in September\r\n, Robinhood announced pla..."
4,"{'id': None, 'name': 'Gizmodo.com'}",Matt Novak,Crypto.com Finally Acknowledges $34 Million St...,Trading platform Crypto.com lost about $34 mil...,https://gizmodo.com/crypto-com-finally-acknowl...,https://i.kinja-img.com/gawker-media/image/upl...,2022-01-20T12:00:00Z,Trading platform Crypto.com lost about $34 mil...


In [36]:
# Describe the Bitcoin Sentiment
# The bitcoin sentiment is neutral with with some positive positive scores

In [37]:
# Describe the Ethereum Sentiment
# The sentiment analysis of eth is neutral with little positive scores 

### Questions:

Q: Which coin had the highest mean positive score?

A: ETH

Q: Which coin had the highest compound score?

A: BTC

Q. Which coin had the highest positive score?

A: BTC

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [38]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [57]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sentence = pd.DataFrame.from_dict(eth_headlines["articles"])
print(sentence)
words = word_tokenize('articles')
print(words)
sw = set(stopwords.words('english'))
first_result = [word.lower() for word in words if word.lower() not in sw]

# Expand the default stopwords list if necessary
sw_addon = {'about', 'to', 'form', 'the', 'and', 'said', 'for', 'that', 'their', 'there', 'it'}
second_result = [word.lower() for word in words if word.lower() not in sw.union(sw_addon)]

                                      source  \
0   {'id': 'the-verge', 'name': 'The Verge'}   
1        {'id': None, 'name': 'Gizmodo.com'}   
2     {'id': None, 'name': 'New York Times'}   
3     {'id': 'engadget', 'name': 'Engadget'}   
4        {'id': None, 'name': 'Gizmodo.com'}   
..                                       ...   
95           {'id': None, 'name': 'Due.com'}   
96         {'id': None, 'name': 'ReadWrite'}   
97      {'id': None, 'name': 'Techmeme.com'}   
98      {'id': None, 'name': 'Slashdot.org'}   
99       {'id': None, 'name': 'Motley Fool'}   

                                               author  \
0                                         Corin Faife   
1                                          Matt Novak   
2   Dana Rubinstein, Jonah E. Bromwich and Jeffery...   
3                                      Igor Bonifacic   
4                                          Matt Novak   
..                                                ...   
95                      

In [71]:
# Complete the `clean_text` function
def clean_text(articles):
    
    # Define a set of stopwords using `stopwords.words()`
    sw = set(stopwords.words('english'))

    # Define the regex parameters
    regex = re.compile("[^a-zA-Z ]")

    # Apply regex parameters to article
    re_clean = regex.sub('', articles)

    # Apply `word_tokenize` to the regex scrubbed text
    words = word_tokenize(re_clean)

    # Create list of lower-case words that are not in the stopword set
    output = [word.lower() for word in words if word.lower() not in sw]
    
    # Return the final list
    return output

SyntaxError: invalid syntax (<ipython-input-71-bdba13019265>, line 2)

In [70]:
result = clean_text(articles)

NameError: name 'articles' is not defined

In [ ]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text

   
    # Create a tokenized list of the words
    
    
    # Lemmatize words into root words

   
    # Convert the words to lowercase
    
    
    # Remove the stop words
    
    
    return tokens

In [ ]:
# Create a new tokens column for Bitcoin
# YOUR CODE HERE!

In [ ]:
# Create a new tokens column for Ethereum
# YOUR CODE HERE!

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [ ]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [ ]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [ ]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [ ]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---